In [1]:
from os import listdir
from pickle import dump
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
import pickle
import os
import cv2
from sklearn.model_selection import train_test_split
%pylab inline
import matplotlib.pyplot as plt
#


/Users/vedparanjape/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [2]:
def extract_features(directory,target_size):

    model = VGG16()
    #Modify model to remove the last layer
    model.layers.pop()
    model = Model(inputs=model.inputs,outputs=model.layers[-1].output)
    print(model.summary())
    
    with open('msft.pickle', 'rb') as handle:
        d = pickle.load(handle)
    
    features = {
        'train_X':[],
        'train_y':[],
        'test_X':[],
        'test_y':[]
    }
    
    train_X = d['train_X']
    train_y = d['train_y']
    test_X = d['test_X']
    test_y = d['test_y']
    print('asas',len(train_X))
    
    for i,img in enumerate(train_X):

        image = cv2.resize(img, dsize=(target_size, target_size), interpolation=cv2.INTER_CUBIC)
        image1 = [image]
        image1.append(image)
        image1.append(image)
        image = np.asarray(image1)
        image = np.transpose(image)
        
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        # get features
        features['train_X'].append(model.predict(image, verbose=0))
        if train_y[i][0] == 1:
            features['train_y'].append(0)
        if train_y[i][1] == 1:
            features['train_y'].append(1)
        if train_y[i][2] == 1:
            features['train_y'].append(2)
    print('Done with training data')
    
    for i,img in enumerate(test_X):

        image = cv2.resize(img, dsize=(target_size, target_size), interpolation=cv2.INTER_CUBIC)
        image1 = [image]
        image1.append(image)
        image1.append(image)
        image = np.asarray(image1)
        image = np.transpose(image)
        
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        # get features
        features['test_X'].append(model.predict(image, verbose=0))
        if test_y[i][0] == 1:
            features['test_y'].append(0)
        if test_y[i][1] == 1:
            features['test_y'].append(1)
        if test_y[i][2] == 1:
            features['test_y'].append(2)
        
    return features
#         dump(features, open("ClientRaw"+".pkl", 'wb'))
        



In [3]:
d = img_features = extract_features("raw",224)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

KeyboardInterrupt: 

In [4]:
print(len(d['train_y']))
print(len(d['test_y']))

598
103


In [5]:
with open("vgg_outs.pickle", "wb") as output_file:
    pickle.dump(d, output_file)